# Create COCO Datasets with xview data

Need to edit the path to images.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import random
import datetime

In [ ]:
from annotation_formatter import LocalizationAnnotationFormatter, DamageAnnotationFormatter

In [ ]:
# http://code.activestate.com/recipes/303060-group-a-list-into-sequential-n-tuples/
def group(lst, n):
    """group([0,3,4,10,2,3], 2) => [(0,3), (4,10), (2,3)]
    
    Group a list into consecutive n-tuples. Incomplete tuples are
    discarded e.g.
    
    >>> group(range(10), 3)
    [(0, 1, 2), (3, 4, 5), (6, 7, 8)]
    """
    return zip(*[lst[i::n] for i in range(n)])

In [ ]:
# each file corresponds to an image
annotation_files = sorted(glob.glob("data/train/labels/*"))
annotation_files_as_pairs = list(group(annotation_files, 2))
random.Random(4).shuffle(annotation_files_as_pairs)
index = int(0.99*len(annotation_files_as_pairs))
# now create the lists
pre_train_files = []
pre_val_files = []
post_train_files = []
post_val_files = []
for (post, pre) in annotation_files_as_pairs[0:index]:
    pre_train_files.append(pre)
    post_train_files.append(post)
for (post, pre) in annotation_files_as_pairs[index:]:
    pre_val_files.append(pre)
    post_val_files.append(post)

In [ ]:
print("LOCALIZATION")
print("Num train files: {}\nNum val files: {}".format(len(pre_train_files), len(pre_val_files)))

In [ ]:
print("DAMAGE")
print("Num train files: {}\nNum val files: {}".format(len(post_train_files), len(post_val_files)))

In [ ]:
# create train and validation datasets for instance segmentation
train_annotation_formatter = LocalizationAnnotationFormatter(instance_segmentation=True)
for filename in pre_train_files:
    train_annotation_formatter.add_image_from_filename(filename)
train_annotation_formatter.write_to_json("./datasets/combined_xview_instance_segmentation_dataset_train.json")

val_annotation_formatter = LocalizationAnnotationFormatter(instance_segmentation=True)
for filename in pre_val_files:
    val_annotation_formatter.add_image_from_filename(filename)
val_annotation_formatter.write_to_json("./datasets/combined_xview_instance_segmentation_dataset_val.json")

In [ ]:
# instance segmentation and damage assessment, train
damage_annotation_formatter = DamageAnnotationFormatter(instance_segmentation=True)
for filename in post_train_files:
    damage_annotation_formatter.add_image_from_filename(filename)
damage_annotation_formatter.write_to_json("./datasets/combined_xview_damage_assessment_instance_segmentation_dataset_train.json")

# instance segmentation and damage assessment, val
damage_annotation_formatter = DamageAnnotationFormatter(instance_segmentation=True)
for filename in post_val_files:
    damage_annotation_formatter.add_image_from_filename(filename)
damage_annotation_formatter.write_to_json("./datasets/combined_xview_damage_assessment_instance_segmentation_dataset_val.json")